[2019 Help with if-like macro](https://discourse.julialang.org/t/help-with-if-like-macro/31856)<br><br>
**Do a dump of your input expression and one of your desired output expression:**

In [8]:
dump(:(@retall if boolex
    a = 6
    b = 7
    c = a+b
    end))

Expr
  head: Symbol macrocall
  args: Array{Any}((3,))
    1: Symbol @retall
    2: LineNumberNode
      line: Int64 1
      file: Symbol e:\JuliaProjects\Training.jl\macro\41.ipynb
    3: Expr
      head: Symbol if
      args: Array{Any}((2,))
        1: Symbol boolex
        2: Expr
          head: Symbol block
          args: Array{Any}((6,))
            1: LineNumberNode
              line: Int64 2
              file: Symbol e:\JuliaProjects\Training.jl\macro\41.ipynb
            2: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Symbol a
                2: Int64 6
            3: LineNumberNode
              line: Int64 3
              file: Symbol e:\JuliaProjects\Training.jl\macro\41.ipynb
            4: Expr
              head: Symbol =
              args: Array{Any}((2,))
                1: Symbol b
                2: Int64 7
            5: LineNumberNode
              line: Int64 4
              file: Symbol e:\JuliaProjects\Training.j

In [9]:
dump(quote 
              a = 6
              b = 7
              c = a+b
              [a,b,c]
              end)

Expr
  head: Symbol block
  args: Array{Any}((8,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol e:\JuliaProjects\Training.jl\macro\41.ipynb
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol a
        2: Int64 6
    3: LineNumberNode
      line: Int64 3
      file: Symbol e:\JuliaProjects\Training.jl\macro\41.ipynb
    4: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol b
        2: Int64 7
    5: LineNumberNode
      line: Int64 4
      file: Symbol e:\JuliaProjects\Training.jl\macro\41.ipynb
    6: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol c
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol +
            2: Symbol a
            3: Symbol b
    7: LineNumberNode
      line: Int64 5
      file: Symbol e:\JuliaProjects\Training.jl\macro\41.ipynb
    8: Expr
      head: Symbol vect
      args: Array{Any}((3,))
        1: Symbol a
        2: 

Then figure out how to transform one nested datatype into the other. (Remember, meta-programming is nothing but nested data-structure gymnastics).<br>
然后找出如何将一种嵌套的数据类型转换为另一种的方法。（记住，元编程不过是嵌套数据结构的体操而已。）

Also, you can enlist the help of **MacroTools.jl**. Although, there is probably nothing wrong with getting your hands dirty (i.e. not use MacroTools) until you’re a bit more experienced.<br>
此外，您还可以使用MacroTools.jl库。不过，如果您的经验还不是很丰富，可能还是亲自动手（即不使用MacroTools）会更好。

In [13]:
ex = :(if 1==1
        "hi"
        3
       end)
propertynames(ex) |> display
ex.args |> display
ex.args[2] |> display
ex.args[2].args |> display

(:head, :args)

2-element Vector{Any}:
 :(1 == 1)
 quote
    #= e:\JuliaProjects\Training.jl\macro\41.ipynb:2 =#
    "hi"
    #= e:\JuliaProjects\Training.jl\macro\41.ipynb:3 =#
    3
end

quote
    #= e:\JuliaProjects\Training.jl\macro\41.ipynb:2 =#
    "hi"
    #= e:\JuliaProjects\Training.jl\macro\41.ipynb:3 =#
    3
end

4-element Vector{Any}:
  :(#= e:\JuliaProjects\Training.jl\macro\41.ipynb:2 =#)
  "hi"
  :(#= e:\JuliaProjects\Training.jl\macro\41.ipynb:3 =#)
 3

In [15]:
macro test(expr)
    expr.args[2] =  expr.args[2].args
    return expr
end
@test if 1==1
    "hi"
    3
end

4-element Vector{Any}:
  :(#= e:\JuliaProjects\Training.jl\macro\41.ipynb:6 =#)
  "hi"
  :(#= e:\JuliaProjects\Training.jl\macro\41.ipynb:7 =#)
 3

In [16]:
macro test(expr)
    expr.args[2] = :(filter(
                            !isnothing, 
                            eval.($(expr.args[2].args))
                           )
                    )
    return expr
 end
 
 @test if 1==1
     "hi"
     3
 end
 #returns ["hi", 3]

2-element Vector{Any}:
  "hi"
 3

In [17]:
macro test(expr)
    expr.args[2] = esc(:(
                            [$([arg for arg in expr.args[2].args if !isa(arg, LineNumberNode)]...),]
                        ))
end
@test if 1==1
    "hi"
    3
end

2-element Vector{Any}:
  "hi"
 3

In [1]:
eval(
    Expr(
            :macro, 

            Expr(:call, :if, :cond, :body), 

            :(
                nothing; 
                esc(Expr(:if, cond, 
                         Expr(:vect, 
                              map(x -> Expr(:block, x), filter(x -> !(x isa LineNumberNode), body.args))...
                             )
                        )
                    )
            )
        )
    )

@if (macro with 1 method)

In [2]:
h1(x) = "<h1> $x " * raw"</h1>"
h2(x) = "<h2> $x " * raw"</h2>"
p(x)  = "<p> $x "  * raw"</p>"

p (generic function with 1 method)

In [3]:
@if true begin
    h1("Hello!")
    h2("Welcome to foo")
    p("Foo is the best")
end

3-element Vector{String}:
 "<h1> Hello! </h1>"
 "<h2> Welcome to foo </h2>"
 "<p> Foo is the best </p>"

In [4]:
@macroexpand @if true begin
    h1("Hello!")
    h2("Welcome to foo")
    p("Foo is the best")
end

:(if true
      [begin
              h1("Hello!")
          end, begin
              h2("Welcome to foo")
          end, begin
              p("Foo is the best")
          end]
  end)

the macro can be written in more julian form as

In [5]:
@eval macro $(Symbol("if"))(cond, body)
    out_body = [Expr(:block, arg) for arg in body.args if !(arg isa LineNumberNode)]
    quote
        if $cond 
            [$(out_body...)]
        end
    end |> esc
end

@if (macro with 1 method)

In [6]:
@macroexpand @if true begin
    h1("Hello!")
    h2("Welcome to foo")
    p("Foo is the best")
end

quote
    #= e:\JuliaProjects\Training.jl\macro\41.ipynb:4 =#
    if true
        #= e:\JuliaProjects\Training.jl\macro\41.ipynb:5 =#
        [begin
                h1("Hello!")
            end, begin
                h2("Welcome to foo")
            end, begin
                p("Foo is the best")
            end]
    end
end

In [7]:
@if true begin
    h1("Hello!")
    h2("Welcome to foo")
    p("Foo is the best")
end

3-element Vector{String}:
 "<h1> Hello! </h1>"
 "<h2> Welcome to foo </h2>"
 "<p> Foo is the best </p>"